You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def get_openai_api_key():
    return os.getenv("OPENAI_API_KEY")
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Title: Unveiling the Power of Artificial Intelligence: Latest Trends, Key Players, and Insights

Introduction:
Artificial Intelligence (AI) is revolutionizing industries worldwide, with continuous advancements reshaping the way we live and work. In this article, we delve into the latest trends in AI, spotlight key players driving innovation, and provide insights to help you navigate this dynamic landscape effectively.

Audience Analysis:
Our target audience includes professionals seeking to expand their knowledge of AI, technology enthusiasts eager to stay updated on the latest trends, and

- Display the results of your execution as markdown in the notebook.

In [15]:
from IPython.display import Markdown
Markdown(result.raw)

# Unveiling the Future of Artificial Intelligence: Trends, Key Players, and How it Impacts You

## Introduction

Artificial Intelligence (AI) stands at the forefront of technological innovation, revolutionizing the way we interact with machines and data. Defined as the simulation of human intelligence processes by machines, AI has become an integral part of our daily lives, from virtual assistants to predictive algorithms. In today's fast-paced world, the importance of AI cannot be understated, as it drives efficiency, enhances decision-making processes, and unlocks new possibilities across various industries.

## Latest Trends in Artificial Intelligence

Machine Learning and Deep Learning are driving the latest advancements in AI, allowing systems to learn from data and improve their performance without being explicitly programmed. These technologies are powering breakthroughs in healthcare, finance, and diverse sectors, enabling personalized medicine, fraud detection, and more. However, with great power comes great responsibility, leading to a growing emphasis on ethical considerations in AI development. Ensuring transparency, fairness, and accountability in AI algorithms is paramount to building trust and mitigating potential biases.

## Key Players in the AI Industry

Tech giants like Google, Amazon, and Microsoft dominate the AI landscape, leveraging their resources and expertise to push the boundaries of innovation. Alongside these industry titans, emerging startups are making significant strides in AI, bringing fresh ideas and disruptive solutions to the market. This diverse ecosystem of players fosters competition, spurring advancements and democratizing access to AI technologies across sectors.

## Noteworthy News on AI

Stay informed with the latest breakthroughs in AI research, from cutting-edge algorithms to AI-driven applications reshaping our future. These developments highlight the transformative power of AI in enhancing efficiency, enabling new services, and addressing complex challenges. By keeping abreast of AI news updates, enthusiasts and professionals can gain valuable insights into the evolving landscape of artificial intelligence.

## Impact of AI on Society

The rise of AI is reshaping the workforce, with automation threatening traditional jobs while creating opportunities for innovative roles. Understanding the implications of AI on employment is crucial for preparing individuals and industries for the future job market. Additionally, AI plays a pivotal role in enhancing customer experiences, from personalized recommendations to predictive analytics, enriching interactions and driving business growth.

## Call to Action

As AI continues to evolve, it is essential for individuals to stay informed about developments, trends, and ethical considerations in the field. Engage with AI news sources such as Forbes, MIT Technology Review, and World Economic Forum to deepen your understanding of the societal impact of AI. Share your thoughts on how AI is shaping our world and participate in the dialogue surrounding its implications.

In conclusion, Artificial Intelligence presents a world of possibilities and challenges, shaping the way we work, interact, and innovate. Embracing the future of AI requires a nuanced understanding of its trends, key players, and societal impact. Let's embark on this journey together, exploring the limitless potential of Artificial Intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [16]:
topic = "Non-profit organizations use cases for AI"
result = crew.kickoff(inputs={"topic": topic})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Non-profit organizations use cases for AI.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Title: Leveraging AI for Social Impact: Non-profit Organizations Utilizing Artificial Intelligence


Introduction:
In recent years, the integration of artificial intelligence (AI) in non-profit organizations has revolutionized the way these entities operate, enabling them to streamline processes, optimize resource allocation, and reach their goals more effectively. This article explores the use cases of AI in the non-profit sector, highlighting the key players, recent trends, and the potential impact of AI technology on social causes.


Key Points:
I. Overview of AI in th

In [17]:
Markdown(result.raw)

# Leveraging AI for Social Impact: Non-profit Organizations Utilizing Artificial Intelligence

## Introduction
In an era characterized by technological advancements, non-profit organizations have increasingly turned to artificial intelligence (AI) to enhance their operations and drive more significant social impact. AI has emerged as a valuable tool that allows non-profits to streamline processes, optimize resource allocation, and achieve their goals more efficiently. This article delves into the various use cases of AI in the non-profit sector, shedding light on how AI technology is transforming social impact initiatives.

Artificial Intelligence, or AI, refers to the simulation of human intelligence processes by machines, including learning, reasoning, and self-correction. In the realm of non-profit organizations, AI plays a critical role in enhancing efficiency and effectiveness. By harnessing AI tools, non-profits can automate tasks, analyze vast amounts of data, and make data-driven decisions that contribute to achieving their social missions. The integration of AI can lead to improved outcomes, increased transparency, and enhanced scalability for non-profit initiatives.

## Overview of AI in the Non-profit Sector
AI integration offers a wide array of benefits for social impact initiatives within non-profit organizations. From enhancing fundraising efforts to improving program evaluation, AI tools empower non-profits to make informed decisions and allocate resources strategically. By leveraging AI for tasks such as donor management, volunteer matching, and predictive analytics, non-profits can optimize their operations and focus on maximizing their impact in the communities they serve.

AI tools can analyze donor behavior, predict giving patterns, and personalize fundraising campaigns to maximize donations. Non-profits can use AI-powered platforms to identify potential donors, tailor communication strategies, and optimize fundraising strategies for greater success. AI enables non-profits to track and evaluate the effectiveness of their programs in real-time. By analyzing data and feedback, AI algorithms can help organizations measure outcomes, assess impact, and make data-informed decisions to continuously improve their initiatives. AI platforms can match volunteers with suitable opportunities based on their skills, interests, and availability. By automating the volunteer matching process, non-profits can increase volunteer engagement, retention, and overall satisfaction, leading to more sustainable volunteer programs. AI-powered predictive analytics tools help non-profits forecast trends, anticipate community needs, and optimize resource allocation. By leveraging data-driven insights, organizations can make informed decisions, mitigate risks, and adapt their strategies to better serve their beneficiaries.

Stay tuned for Part 2 of the blog post.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).